## Gaussian pulse moving through dielectric $\varepsilon$ and absorbing $\sigma$ layers

Study the optimal values of PML's for different frequencies and PML widths to minimize reflection:

⚠️ work in progress!

![](https://codimd.web.cern.ch/uploads/upload_377b25730e2b4ec04c803bbab82562da.gif)

### Simulation with wakis
----

In [1]:
from scipy.constants import c as c_light
import numpy as np
from tqdm import tqdm

# import wakis package (local)
import sys
sys.path.append('../wakis')
from wakis.solverFIT3D import SolverFIT3D
from wakis.gridFIT3D import GridFIT3D
from wakis.sources import WavePacket

# import wakis package from pip
#from wakis import SolverFIT3D, GridFIT3D
#from wakis.sources import WavePacket

# Plotting settings
import pyvista as pv
import matplotlib.animation
import matplotlib.pyplot as plt
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.ioff()

%matplotlib ipympl

### Domain definition and absorbing barriers

In [2]:
# barrier dimensions
lx, ly, lz = 5., 5., 10.

# Domain bounds
xmin, xmax, ymin, ymax, zmin, zmax = -lx/2., lx/2., -ly/2., ly/2., -lz/2.,lz/2.

# Number of mesh cells
Nx = 4
Ny = 4
Nz = 800

# cell size
dx, dy, dz = lx/Nx, ly/Ny, lz/Nz

# Barriers
barrier = {}
stl_solids = {}
stl_materials = {}

n = 16
ss = np.linspace( np.sqrt(1.e-4), np.sqrt(1.e-1), n, )
ss = ss*ss

width = 2.*dz
pos_z_begin = zmax - n*width

for k, si in enumerate(ss):
    
    name = f'b{k}'
    file_name = f'barrier{k}.stl'
    
    barrier[k] = pv.Cube( x_length=lx, y_length=ly, z_length=width, center=(0,0,pos_z_begin+k*width))
    barrier[k].save( file_name, )

    stl_solids[name] = file_name
    stl_materials[name] = [ 1., 1., si, ]

In [ ]:
# Simulation grid
grid = GridFIT3D(xmin, xmax, ymin, ymax, zmin, zmax, 
                 Nx, Ny, Nz, 
                 stl_solids=stl_solids, 
                 stl_materials=stl_materials)
grid.inspect()

### Solver setup

In [ ]:
# boundary conditions
bc_low=['periodic', 'periodic', 'pec']  
bc_high=['periodic', 'periodic', 'pec']

solver = SolverFIT3D(grid=grid,     # pass grid object
                     cfln=0.50,       # Default if no dt is defined
                     bc_low=bc_low, 
                     bc_high=bc_high, 
                     use_stl=True,     # Enables or disables geometry import
                     bg='vacuum',      # Background material 
                     use_gpu=False,
                     )


### Source setup

In [ ]:
# Add Gaussian wavepacket
source = WavePacket(xs=slice(0,Nx), ys=slice(0,Ny),
                    sigmaxy=100.,
                    sigmaz=.4,
                    wavelength=16.*dz,
                    tinj=2.0)

t = np.arange(0,(zmax-zmin)/c_light, solver.dt)
source.plot( t, )

# plot 2d
X, Y = np.meshgrid(solver.x, solver.y)
gaussxy = np.exp(-(X**2+Y**2)/(2*source.sigmaxy**2))
#plt.figure()
#plt.contourf(X,Y,gaussxy)

### Time-stepping

In [ ]:
#initialize to 0
for d in ['x','y','z']:
    solver.E[:,:,:,d] = 0.
    solver.H[:,:,:,d] = 0.
    solver.J[:,:,:,d] = 0.

# Fields to save
Ex, Ey, Hx, Hy, Jz = [], [], [], [], []

plot2D=False
plot3D=False

N1 = 5*int((2+zmax-zmin)/c_light/solver.dt)
for n in tqdm(range(N1)):

    #apply source
    source.update(solver, solver.dt*n)

    #Advance fields
    solver.one_step()

    if n%100 == 0:
        Ex.append(solver.E[Nx//2, Ny//2, :, 'x'])
        Ey.append(solver.E[Nx//2, Ny//2, :, 'y'])
        Hx.append(solver.H[Nx//2, Ny//2, :, 'x'])
        Hy.append(solver.H[Nx//2, Ny//2, :, 'y'])
        Jz.append(solver.J[Nx//2, Ny//2, :, 'z'])

    if plot2D and n%50 == 0:
        solver.plot2D(field='H', component='y', plane='ZY', pos=0.5, norm='symlog', 
               vmin=-1, vmax=1, figsize=[8,4], cmap='RdBu', patch_alpha=0.1, 
               add_patch=None, title='Hy', off_screen=True, n=n, interpolation='spline36')
    if plot3D and n==100:
        solver.plot3D( 'Hy',n=n,  add_stl='barriers', clip_interactive=True, )
    

### Plotting and animations 

In [9]:
# quick plotting
solver.plot3D( 'Hy', add_stl='barrier1', clip_interactive=True, stl_opacity=0.4, cmap='gnuplot' )

In [7]:
from argparse import Namespace as NS
fields = { 
  'Ex': NS( values=np.array(Ex), units='', ), 
  'Ey': NS( values=np.array(Ey), units='', ), 
  'Hx': NS( values=np.array(Hx), units='', ),
  'Hy': NS( values=np.array(Hy), units='', ),
  'Jz': NS( values=np.array(Jz), units='', ),
}

# frames animation
fig, axs = plt.subplots( len(fields), 1, figsize=[12,8], dpi=150, )

axs[-1].set_xlabel(f'z [m]')
for ax, ( ki, vi, ) in zip(axs,fields.items()):

  ax.set_ylabel(f'Field {ki} {vi.units}')

  q = np.abs(vi.values).max()
  ax.set_ylim((-q,q))
  ax.set_xlim((zmin,zmax))
  
  # barriers
  for b, si in zip(barrier.values(),ss):
    xlo, xhi, ylo, yhi, zlo, zhi = b.bounds
    ax.axvspan(zlo, zhi, color='g', alpha=.1 + .6*si/ss.max())

  ls = [ ax.plot(solver.z, np.full_like( solver.z, np.nan, ), c='r')[0] for ax in axs ]

def animate(frame,ls,fields,axs):
  fig.gca()
  axs[0].set_title(f'frame {frame:6d}')
  # field
  for li, ( ki, vi, ) in zip(ls,fields.items()):
    li.set( data=(solver.z,vi.values[frame]), )
  return ls

anim = matplotlib.animation.FuncAnimation(fig, lambda frame: animate(frame,ls,fields,axs), frames=len(vi.values), )
anim.save(f'movie_all.mp4')
plt.close(fig)

### Metrics to analyise absortion

In [ ]:
def metric_area( f, ):
    return np.sum(np.abs(f))

def metric_length( f, ):
    df = np.diff(f)
    return np.sum(np.sqrt(1. + df*df))/len(df) - 1

for vi in fields.values():
    vi.area = np.array([ metric_area(fs) for fs in vi.values ])
    vi.area /= vi.area.max()
    vi.length = np.array([ metric_length(fs) for fs in vi.values ])
    vi.length /= vi.length.max()

In [ ]:
fig, ( ax1, ax2, ) = plt.subplots( 1, 2, tight_layout=True, )

for ki, vi in fields.items():
    ax1.plot( vi.area, label=ki, )
    ax2.plot( vi.length, label=ki, )

ax1.legend(), ax2.legend()

In [ ]:
plt.figure()
plt.imshow(solver.E[Nx//2, :, :, 'y'], aspect='auto')
plt.colorbar()
plt.show()

In [ ]:
plt.figure()
plt.imshow(solver.H[Nx//2, :, :, 'x'], aspect='auto')
plt.colorbar()
plt.show()